# <font color='Red'>Basic two-phase compositional model</font>

## <font color='blue'>Introduction</font>

#### In the first case study, we cover the basic structures and procedures to run simulation with DARTS. 
#### DARTS contains a main module engine: 
* <font color='red'>engines</font>  
    
#### This module is programmed in C++ and exposed to python with compiled packages (libraries) named as
* <font color='red'>engines.pyd</font>
  
#### The module engines.pyd provides the run-simulation needed functionalities, like
* reservoir (mesh) initialization
* well settings
* interpolation operation
* jacobian assembly
* linear and nonlinear iteration

#### The module engines.pyd can be found in the folder <font color='red'>'darts'</font>.

## <font color='blue'>The objectives for the first exercise</font>
1. Be familiar with the basic procedures to run a simulation
2. Exercise on several simulation parameters including timestep controls and mesh 

## 1D Model can be divided into following parts:
<img style="float: left;" src="slides/Slide4.PNG" width="60%">

## <font color='blue'>Let's start !</font>
### Step 1. We need to import <font color='red'>engines</font> and nessesary physical properties into the workspace, just like the import of commonly-used modules such as numpy etc.

In [ ]:
'''Import all important packages from DARTS installation'''
from darts.engines import *
from darts.models.physics_sup.property_container import property_container
from darts.models.physics_sup.physics_comp_sup import Compositional
from darts.models.physics_sup.properties_basic import Density, ViscosityConst, PhaseRelPerm, Flash

import numpy as np
import matplotlib.pyplot as plt

### Step 2. We need to specify the time recorder, which could help to make sure about the performance of the simulator in different parts. Timers can be created in hierachical order.

<img style="float: left;" src="slides/Slide6.PNG" width="60%">

In [ ]:
'''Activate main timers for simulation'''
def activate_timer():
    # Call class constructor and Build timer_node object
    timer = timer_node()

    # Call object members; there are 2 types of members:
    ## <1>Function member:
    timer.start()
    ## <2>Data member:
    timer.node["simulation"] = timer_node()
    timer.node["initialization"] = timer_node()

    # Start initialization
    timer.node["initialization"].start()
    
    return timer

### Step 3. Simulation parameters setting. Here we specify the simulation parameters, e.g.:

* timestep strategy (first_ts, max_ts, mult_ts)
* convergence criteria (tolerance of nonlinear iteration and other parameters controlling iterations)

<img style="float: left;" src="slides/Slide7.PNG" width="60%">

In [ ]:
'''Define main parameters for simulation by overwriting default parameters'''
def define_params(max_ts=1):
    # Build a sim_params object by calling constructor
    params = sim_params()

    # Adjust time step settings
    # First time step (time unit: day)
    params.first_ts = 0.01

    # Time step multiplier
    params.mult_ts = 4

    # Maximum time step
    params.max_ts = max_ts

    # Newton tolerance
    params.tolerance_newton = 1e-10
    
    return params

### Step 4. Mesh initialization
<img style="float: left;" src="slides/Slide8.PNG" width="60%">

<img style="float: left;" src="slides/Slide9.PNG" width="60%">

In [ ]:
'''Mesh initialization for 1D reservoir with constant transmissibility'''
def init_mesh(nb):
    # Create mesh object by calling the class constructor
    mesh = conn_mesh()

    # Create connection list for 1D reservoir 
    block_m = np.arange(nb - 1, dtype=np.int32)
    block_p = block_m + 1

    # Set constant transmissbility
    permeability = 2
    tranD = np.ones(nb - 1) * 1e-3 * nb 
    tran = tranD * permeability

    # Initialize mesh with connection list
    mesh.init(index_vector(block_m), index_vector(block_p),
              value_vector(tran), value_vector(tranD))

    # Complete mesh initialization
    mesh.reverse_and_sort()
    
    return mesh

### Step 5. Define reservoir properties
* connecting numpy arrays to the mesh
* fill porosity, depth, volume, conduction and heat capacity
* imitate boundary conditions with large volumes

<img style="float: left;" src="slides/Slide10.PNG" width="60%">

In [ ]:
'''Define basic properties for the reservoir'''
def define_reservoir(nb):
    # Create numpy arrays wrapped around mesh data (no copying)
    volume = np.array(mesh.volume, copy=False)
    porosity = np.array(mesh.poro, copy=False)
    depth = np.array(mesh.depth, copy=False)

    # Assign volume, porosity and depth values
    volume.fill(3000 / nb)
    porosity.fill(0.2)
    depth.fill(1000)

    # Make first and last blocks large (source/sink)
    volume[0] = 1e10
    volume[nb-1] = 1e10

### Step 6. Boundary and initial conditions
* initial reservoir conditions
* pressure source and sink
* translation from temperature to enthalpy

<img style="float: left;" src="slides/Slide11.PNG" width="60%">

In [ ]:
'''Mimic boundary conditions for the reservoir'''
def define_initial_conditions(nb):
    # Create numpy wrappers for initial solution
    pressure = np.array(mesh.pressure, copy=False) 
    fraction = np.array(mesh.composition, copy=False)

    # Assign initial pressure values
    pressure.fill(200)
    pressure[0] = 250
    pressure[nb-1] = 150

    # Assign molar fraction values
    fraction.fill(0.1)
    fraction[0] = 0.9

### Step 7. Physics initialization
* translate temperature range to enthalpy
* initialize physical ranges

<img style="float: left;" src="slides/Slide12.PNG" width="60%">

In [ ]:
'''Create physics from predefined properties from DARTS package'''
def define_physics():
    # basic physical parameters
    zero = 1e-8
    components_name = ['CO2', 'H2O']
    Mw = [44.01, 18.015]
    
    # activate property container
    property = property_container(phases_name=['gas', 'oil'],
                                  components_name=components_name,
                                  Mw=Mw, min_z=zero/10)

    # properties correlations
    property.flash_ev = Flash(components_name, [4.0, 0.2], zero)
    property.density_ev = dict([('gas', Density(compr=1e-3, dens0=200)),
                                ('oil', Density(compr=1e-5, dens0=600))])
    property.viscosity_ev = dict([('gas', ViscosityConst(0.05)),
                                  ('oil', ViscosityConst(0.5))])
    property.rel_perm_ev = dict([('gas', PhaseRelPerm("gas")),
                                 ('oil', PhaseRelPerm("oil"))])

    # Activate physics
    physics = Compositional(property, timer, n_points=200, min_p=100, max_p=300,
                            min_z=zero/10, max_z=1-zero/10)

    return physics

### Step 8. Engine initialization and run
<img style="float: left;" src="slides/Slide13.PNG" width="60%">

In [ ]:
# create all model parameters
nb = 50
timer = activate_timer()
params = define_params(max_ts=1)
mesh = init_mesh(nb)
define_reservoir(nb)
define_initial_conditions(nb)
physics = define_physics()

# Initialize engine
physics.engine.init(mesh, ms_well_vector(),
                    op_vector([physics.acc_flux_itor[0]]),
                    params, timer.node["simulation"])

# Stop initialization timer
timer.node["initialization"].stop()

# Run simulator for 500 days
physics.engine.run(500)

# Print timers (note where most of the time was spent!)
print(timer.print("", ""))

### Step 9. Data processing and plots

<img style="float: left;" src="slides/Slide14.PNG" width="60%">

In [ ]:
# Define function to plot data profiles
%matplotlib inline
def plot_profile(data, name, sp, ax):
    n = len(data)    
    ax.plot(np.arange(n), data[0:n], '-')
    ax.set_xlabel('Grid index')
    ax.set_ylabel('%s' % (name))

In [ ]:
# Get numpy wrapper for final solution
X = np.array(physics.engine.X, copy=False)

# Prepare for plotting
fig = plt.figure()   

nc = 2

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
# Plot pressure profile
plot_profile(X[0:nc*nb:nc],'Pressure, bar', 1, axes[0])
# Plot molar fraction profile
plot_profile(X[1:nc*nb:nc],'Molar fraction', 2, axes[1])